In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [4]:
# Read the SQLite database from '../DB/time_series.sqlite'
import sqlite3

conn = sqlite3.connect('../DB/time_series.sqlite')

# Give the informations of the DB
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

for table_name in tables:
    print(table_name[0])

time_series_15min_singleindex
time_series_30min_singleindex
time_series_60min_singleindex
